In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

Verified

In [ ]:
import requests
import pandas as pd
import time
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Parameters
window_size = 50  # Number of most recent prices to keep in the window
interval = 60  # Fetch new price every 60 seconds
api_key = os.getenv("API_KEY")  # Finnhub API key

def fetch_latest_price(ticker):
    """Fetch the latest stock price from Finnhub API."""
    url = f"https://finnhub.io/api/v1/quote?symbol={ticker}&token={api_key}"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raises an HTTPError for bad responses
        data = response.json()
        
        if 'c' in data:
            latest_price = data['c']  # Current price
            return time.strftime('%Y-%m-%d %H:%M:%S'), latest_price
        else:
            print(f"Error fetching data for {ticker}: {data.get('error', 'Unknown error')}")
            return None, None
    except requests.RequestException as e:
        print(f"Failed to get ticker '{ticker}' due to: {e}")
        return None, None

def get_valid_ticker():
    """Ask the user for a valid ticker until one is provided."""
    while True:
        ticker = input("Please enter a valid stock ticker: ").upper()
        timestamp, price = fetch_latest_price(ticker)
        if price is not None:
            print(f"Valid ticker '{ticker}' provided.")
            return ticker
        else:
            print(f"'{ticker}' is not a valid stock ticker or data could not be retrieved. Please try again.")

def update_csv(stock_data, csv_file, ticker):
    """Update the CSV file with a new row and maintain a moving window."""
    timestamp, price = fetch_latest_price(ticker)
    
    if price is not None:
        new_data = pd.DataFrame({"Datetime": [timestamp], "Price": [price]})
        stock_data = pd.concat([stock_data, new_data], ignore_index=True).tail(window_size)
        stock_data.to_csv(csv_file, index=False)
        print(f"Updated CSV at {timestamp} with price {price}")
    else:
        print(f"Failed to fetch data for ticker {ticker}")

    return stock_data

# Main script
if __name__ == "__main__":
    # Get the current working directory (notebooks)
    current_dir = os.getcwd()

    # Go up one level to the parent directory
    parent_dir = os.path.dirname(current_dir)

    # Construct the path to the stock_data directory
    stock_data_dir = os.path.join(parent_dir, "stock_data")

    # Create the stock_data directory if it doesn't exist
    os.makedirs(stock_data_dir, exist_ok=True)

    ticker = get_valid_ticker()
    csv_file = os.path.join(stock_data_dir, f"{ticker}.csv")

    # Initialize or load existing CSV file
    try:
        stock_data = pd.read_csv(csv_file)
    except FileNotFoundError:
        stock_data = pd.DataFrame(columns=["Datetime", "Price"])

    # Main loop to continuously fetch data and update the CSV
    while True:
        stock_data = update_csv(stock_data, csv_file, ticker)
        time.sleep(interval)

Verified

In [ ]:
import requests
import pandas as pd
import time
import os
from dotenv import load_dotenv
from datetime import datetime, time as dt_time
from pytz import timezone
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay

# Load environment variables
load_dotenv()

# Parameters
window_size = 50  # Number of most recent prices to keep in the window
interval = 60  # Fetch new price every 60 seconds
api_key = os.getenv("API_KEY")  # Finnhub API key

def is_trading_time():
    now = datetime.now(timezone('US/Eastern'))
    
    # Check if it's a weekday
    if now.weekday() >= 5:  # 5 = Saturday, 6 = Sunday
        return False
    
    # Check if it's a holiday
    us_holidays = CustomBusinessDay(calendar=USFederalHolidayCalendar())
    if now.strftime('%Y-%m-%d') in us_holidays.holidays():
        return False
    
    # Check if it's within market hours (9:30 AM to 4:00 PM Eastern Time)
    market_start = dt_time(9, 30)
    market_end = dt_time(16, 0)
    current_time = now.time()
    
    return market_start <= current_time <= market_end

def fetch_latest_price(ticker):
    """Fetch the latest stock price from Finnhub API."""
    url = f"https://finnhub.io/api/v1/quote?symbol={ticker}&token={api_key}"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raises an HTTPError for bad responses
        data = response.json()
        
        if 'c' in data:
            latest_price = data['c']  # Current price
            return time.strftime('%Y-%m-%d %H:%M:%S'), latest_price
        else:
            print(f"Error fetching data for {ticker}: {data.get('error', 'Unknown error')}")
            return None, None
    except requests.RequestException as e:
        print(f"Failed to get ticker '{ticker}' due to: {e}")
        return None, None

def get_valid_ticker():
    """Ask the user for a valid ticker until one is provided."""
    while True:
        ticker = input("Please enter a valid stock ticker: ").upper()
        timestamp, price = fetch_latest_price(ticker)
        if price is not None:
            print(f"Valid ticker '{ticker}' provided.")
            return ticker
        else:
            print(f"'{ticker}' is not a valid stock ticker or data could not be retrieved. Please try again.")

def update_csv(stock_data, csv_file, ticker):
    """Update the CSV file with a new row and maintain a moving window."""
    if not is_trading_time():
        print(f"Not trading time. Skipping update at {datetime.now(timezone('US/Eastern'))}")
        return stock_data

    timestamp, price = fetch_latest_price(ticker)
    
    if price is not None:
        new_data = pd.DataFrame({"Datetime": [timestamp], "Price": [price]})
        stock_data = pd.concat([stock_data, new_data], ignore_index=True).tail(window_size)
        stock_data.to_csv(csv_file, index=False)
        print(f"Updated CSV at {timestamp} with price {price}")
    else:
        print(f"Failed to fetch data for ticker {ticker}")

    return stock_data

# Main script
if __name__ == "__main__":
    # Get the current working directory (notebooks)
    current_dir = os.getcwd()

    # Go up one level to the parent directory
    parent_dir = os.path.dirname(current_dir)

    # Construct the path to the stock_data directory
    stock_data_dir = os.path.join(parent_dir, "stock_data")

    # Create the stock_data directory if it doesn't exist
    os.makedirs(stock_data_dir, exist_ok=True)

    ticker = get_valid_ticker()
    csv_file = os.path.join(stock_data_dir, f"{ticker}.csv")

    # Initialize or load existing CSV file
    try:
        stock_data = pd.read_csv(csv_file)
    except FileNotFoundError:
        stock_data = pd.DataFrame(columns=["Datetime", "Price"])

    # Main loop to continuously fetch data and update the CSV
    while True:
        stock_data = update_csv(stock_data, csv_file, ticker)
        time.sleep(interval)

Valid ticker 'AAPL' provided.
Updated CSV at 2024-10-24 23:14:28 with price 230.57


C:\Users\rauna\AppData\Local\Temp\ipykernel_34204\2451692705.py:50: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  stock_data = pd.concat([stock_data, new_data], ignore_index=True).tail(window_size)


Untested

In [ ]:
# Regression, Derivation, Math + Notification Systems

Undeveloped